In [1]:
import pandas as pd
import numpy as np
import json
from collections import defaultdict

In [2]:
pd.set_option('display.max_rows', 10)

In [3]:
df = pd.read_csv('ijcai2016_taobao.csv', sep=',')
len(df)

FileNotFoundError: [Errno 2] No such file or directory: 'ijcai2016_taobao.csv'

In [4]:
click_df = df[df['act_ID']==0]
# 0” denotes “click” while “1” for “buy”
len(click_df)

35179371

In [5]:
click_df = click_df.drop_duplicates()
uit_df = click_df[['use_ID','ite_ID','time']]
len(uit_df)

27500939

In [6]:
month_uit_df = uit_df[uit_df['time']<=20150819]
sort_uit_df = month_uit_df.sort_values(by=['use_ID','time'],ascending=[True,True]) 

In [8]:
# 去除交互少于N的user和item
ui_cnt = sort_uit_df['use_ID'].value_counts()
u_index = ui_cnt[ui_cnt>=43].index
ucut_df = sort_uit_df[sort_uit_df['use_ID'].isin(u_index)]

iu_cnt = ucut_df['ite_ID'].value_counts()
i_index = iu_cnt[iu_cnt>=5].index
uicut_df = ucut_df[ucut_df['ite_ID'].isin(i_index)]
uicut_df

,use_ID,ite_ID,time
32864073,48,777353,20150701
32894869,48,1893320,20150701
33348718,48,2048619,20150701
34098119,48,1056298,20150701
44035967,48,280273,20150702
...,...,...,...
18616208,2063585,1373150,20150803
9439292,2063585,2252494,20150804
13972481,2063585,1242815,20150804
33338211,2063585,73623,20150804


In [21]:
uicut_df.nunique()

use_ID     54705
ite_ID    227096
time          50
dtype: int64

In [23]:
utgroup = uicut_df.groupby(['use_ID','time'])
ut_c = utgroup.ite_ID.count()[lambda x: x >= 5]
utc_id = ut_c.index.levels
utc_id_index = ut_c.index.codes

In [24]:
user_session = {}
L = len(utc_id_index[0])
for i in range(L): 
    loc_i = utc_id_index[0][i]
    loc_j = utc_id_index[1][i]
    uid = utc_id[0][loc_i]
    tid = utc_id[1][loc_j]
    if uid not in user_session:
        user_session[uid] = []
    user_session[uid].append(tid)
    
len(user_session)

53796

In [25]:
df_cut=pd.DataFrame(columns=uicut_df.keys())
df_cut

,use_ID,ite_ID,time


In [27]:
# 已经去除了session数量小于2的用户
cnt = 0
short_c = 0
for uid in user_session: 
    if len(user_session[uid])<4:
        short_c+=1
        continue
    cnt+=1
    if cnt%1000==0:
        print("cnt = ",cnt)
    tid = user_session[uid]
    df_oneuser = uicut_df[uicut_df["use_ID"]==uid]
#     df_oneuser
    df_row = df_oneuser[df_oneuser["time"].isin(tid)]
    df_cut = pd.concat([df_cut, df_row],ignore_index=True)

cnt =  1000
cnt =  2000
cnt =  3000
cnt =  4000
cnt =  5000
cnt =  6000
cnt =  7000
cnt =  8000
cnt =  9000
cnt =  10000
cnt =  11000
cnt =  12000
cnt =  13000
cnt =  14000
cnt =  15000
cnt =  16000
cnt =  17000
cnt =  18000
cnt =  19000
cnt =  20000
cnt =  21000
cnt =  22000
cnt =  23000
cnt =  24000
cnt =  25000
cnt =  26000
cnt =  27000
cnt =  28000
cnt =  29000
cnt =  30000


In [ ]:
df_cut.to_csv('Tmall16_m78.csv',index=0)

In [28]:
df_cut=df_cut.drop_duplicates()
df_cut.nunique()

use_ID     30846
ite_ID    224185
time          50
dtype: int64

In [ ]:
# df_cut[["time"]] = df_cut[["time"]]-20150700

In [ ]:
ui_array = df_cut[['use_ID','ite_ID']].values
uid_turn = defaultdict(int)
iid_turn = defaultdict(int)
uid_cnt = 0
# uid_cnt = 1
# 因为用了np.zero生产数据,第0行也要有实际数据,所以uid要从0开始
# maskid是0，对应的是item，userid可以是0
iid_cnt = 1
for row in ui_array:
    uid = row[0]
    iid = row[1]
    if uid not in uid_turn:
        uid_turn[uid] = uid_cnt
        uid_cnt += 1
    if iid not in iid_turn:
        iid_turn[iid] = iid_cnt
        iid_cnt += 1

len(uid_turn)
len(iid_turn)

In [ ]:
uit_value = uit_df.values
userList = {}
# sessionList = {}
for row in uit_value:
    uid = uid_turn[row[0]]
    iid = iid_turn[row[1]]
#     uid = row[0]
#     iid = row[1]
    tid = row[2]
    if uid not in userList:
        userList[uid] = []
        
    userList[uid].append([iid,tid])

#     if tid not in sessionList:
#         userList.append()
#         sessionList[tid] = []
#     sessionList[tid].append(iid)

In [ ]:
ut_List = {}
for uid in userList:
    it_list = userList[uid]
    sessionList = {}
    for row in it_list:
        iid = row[0]
        tid = row[1]
        if tid not in sessionList:
            sessionList[tid] = []
        
        sessionList[tid].append(iid)
    
    ut_List[uid] = sessionList
        

In [ ]:
u_List = {}
for uid in ut_List:
    u_List[uid] = []
    for tid in ut_List[uid]:
        u_List[uid].append(ut_List[uid][tid])

In [ ]:
json_u_List = json.dumps(u_List)
saveFile(json_u_List,'Tmall16.json')